# **베스트 모델 만들기**
### 모델을 업데이트한다.
- 모델을 저장하고 재사용한다. (save(), load_model())
- 이때 그냥 저장하는 것이 아니라, epoch마다 모델의 정확도를 함께 기록하면서 저장한다.
- 현재 디렉토리 안의 model 폴더 안에, epoch 횟수와 테스트 셋 오차 값을 이용해 파일 이름을 만들어 hdf5라는 확장자로 저장한다.

    ex. epoch 100, 오차 0.0612 -> 100-0.0612.hdf5

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorflow==2.0

     |████████████████████████████████| 86.3MB 50kB/s 
     |████████████████████████████████| 3.8MB 53.1MB/s 
     |████████████████████████████████| 450kB 58.4MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [3]:
# TF 2.0 must be installed manually as the default version of TF in Colab is yet 1.15.0.
import tensorflow as tf
print(tf.__version__)

2.0.0


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os # 모델 저장을 위해

Using TensorFlow backend.


In [0]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [6]:
df_pre = pd.read_csv('/content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/dataset/wine.csv', header=None)
df_pre.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


**sample()** 

sample() 함수를 사용하여, 원본 데이터의 몇 %를 사용할지 지정한다. 

sample() 함수는 원본 데이터에서 정해진 비율만큼 랜덤으로 데이터를 뽑아오는 함수로 frac = 1이라면 원본 데이터의 100%를 불러오는 의미, 0.5라면 50%를 불러오는 의미이다.


In [7]:
df = df_pre.sample(frac=1) # 1이면 원본 데이터의 100%를 불러왔다는 의미. (굳이 왜?)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
5316,6.3,0.18,0.24,3.4,0.053,20.0,119.0,0.99373,3.11,0.52,9.2,6,0
5210,6.8,0.14,0.18,1.4,0.047,30.0,90.0,0.99164,3.27,0.54,11.2,6,0
3518,7.3,0.22,0.50,13.7,0.049,56.0,189.0,0.99940,3.24,0.66,9.0,6,0
1622,7.6,0.67,0.14,1.5,0.074,25.0,168.0,0.99370,3.05,0.51,9.3,5,0
2443,7.3,0.21,0.29,1.6,0.034,29.0,118.0,0.99170,3.30,0.50,11.0,8,0


In [8]:
# 0~11까지의 12개 속성으로 13번째 클래스를 맞추는 과제
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 5316 to 2732
Data columns (total 13 columns):
0     6497 non-null float64
1     6497 non-null float64
2     6497 non-null float64
3     6497 non-null float64
4     6497 non-null float64
5     6497 non-null float64
6     6497 non-null float64
7     6497 non-null float64
8     6497 non-null float64
9     6497 non-null float64
10    6497 non-null float64
11    6497 non-null int64
12    6497 non-null int64
dtypes: float64(11), int64(2)
memory usage: 710.6 KB


In [0]:
dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

In [0]:
# 모델 설정
model = Sequential()
model.add(Dense(30,  input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일. 이항분류 문제이므로 binary_crossentropy 오차함수 사용
model.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

**ModelCheckpoint()**

In [11]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건
# checkpointer: 모니터할 값 지정. val_loss는 테스트 오차 (cf. acc: 학습 정확도, val_acc: 테스트 셋 정확도, loss: 학습 셋 오차)
# filepath를 modelpath로 지정. verbose: 1이면 해당 함수의 진행 사항 출력, 0이면 출력되지 않음.
# save_best_only: 앞서 저장한 모델보다 나아졌을 때만 저장
modelpath="/content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 모델 실행 및 저장
# 모델을 학습할 때마다 위에서 정한 checkpointer의 값을 받아 지정된 곳에 모델을 저장함.
model.fit(X, Y, validation_split=0.2, epochs=200, batch_size=200, verbose=0, callbacks=[checkpointer])


Epoch 00001: val_loss improved from inf to 0.37138, saving model to /content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/01-0.3714.hdf5

Epoch 00002: val_loss improved from 0.37138 to 0.27502, saving model to /content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/02-0.2750.hdf5

Epoch 00003: val_loss improved from 0.27502 to 0.23700, saving model to /content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/03-0.2370.hdf5

Epoch 00004: val_loss improved from 0.23700 to 0.22310, saving model to /content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/04-0.2231.hdf5

Epoch 00005: val_loss improved from 0.22310 to 0.21644, saving model to /content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/project/model/05-0.2164.hdf5

Epoch 00006: val_loss did not improve from 0.21644

Epoch 00007: val_loss improved from 0.21644 to 0.20515, saving model to /content/drive/My Drive/Colab Note